# DSCI 525 - Web and Cloud Computing

## Milestone 1: Tackling big data on your laptop

## Overall project goal and data 

During this course, you will be working on a team project involving big data. The purpose is to get exposure to working with much larger datasets than you have previously in MDS. You have been assigned to teams of three or four. (See group assignment in [Canvas](https://canvas.ubc.ca/courses/106517). Unlike previous project courses, in this course, all of you will be working on **the same problem**. In particular, you will be building and deploying ensemble machine learning models in the cloud to predict daily rainfall in Australia on a large dataset (~6 GB), where features are outputs of different climate models, and the target is the actual rainfall observation.  

You will be using [this dataset on figshare](https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681). This folder has the output of different climate models as features, and our ultimate goal is to build an ensemble model on these outputs and compare the results with the actual rainfall. At the end of the project, you should have your ML model deployed in the cloud for others to use. 

During this course, you will work towards this goal step by step in four milestones.  

<br><br>

## Milestone 1 checklist  



### 1. Team-work contract
rubric={correctness:10}

Similar to what you did in DSCI 522 and DSCI 524, create a teamwork contract. The contract should outline how you are committed to working together so that you are accountable to one another. Again, you may start with your team contract document from previous project courses and adapt it to your new team. It is a fairly personal document, and please do not push it into your public repositories. Instead, save it somewhere your team can easily share it, and you can share a link to it or a copy with us in your submission to Canvas to prove you did this.

### 2. Creating a repository and project structure 
rubric={mechanics:10}

1. Similar to previous project courses, create a public repository under [UBC-MDS org](https://github.com/UBC-MDS) for your project. 
2. Write a brief introduction of the project in the `README`. 
3. Create a folder called `notebooks` in the repository and create a notebook for this milestone in that folder.

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from IPython.display import display
import pyarrow.dataset as ds
import pyarrow as pa
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

### 3. Downloading the data 
rubric={correctness:10}

1. Download the data from [figshare](https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681) to your local computer using the [figshare API](https://docs.figshare.com) (you need to make use of `requests` library).

2. Extract the zip file, again programmatically, similar to how we did it in class. 

>  You can download the data and unzip it manually. But we learned about APIs, so we can do it in a reproducible way with the `requests` library, similar to how we [did it in class](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture1.html#using-rest-api-lab-lecture).

> There are 5 files in the figshare repo. The one we want is: `data.zip`

In [2]:
# References code from Lecture notes: https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture1.html#using-rest-api-lab-lecture
# article with daily rainfall data
article_id = 14096681  
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare_data/"

In [3]:
# Get request to get the files available
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  
files = data["files"]          
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
%%time

# Download data.zip
files_to_dl = ["data.zip"] 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 3.35 s, sys: 5.18 s, total: 8.54 s
Wall time: 2min 21s


In [5]:
%%time

# Extract Data.zip
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 7.41 s, sys: 906 ms, total: 8.32 s
Wall time: 12.4 s




### 4. Combining data CSVs
rubric={correctness:10,reasoning:10}

1. Combine data CSVs into a single CSV using pandas.
    
2. When combining the CSV files, add an extra column called "model" that identifies the model.
    Tip 1: you can get this column populated from the file name, eg: for file name "SAM0-UNICON_daily_rainfall_NSW.csv", the model name is SAM0-UNICON
    Tip 2: Remember how we added "year" column when we combined airline CSVs. Here the regex will be to get word before an underscore ie, "/([^_]*)"

> Note: There is a file called `observed_daily_rainfall_SYD.csv` in the data folder that you downloaded. Make sure you exclude this file (programmatically or just take out that file from the folder) before you combine CSVs. We will use this file in our next milestone.

3. ***Compare*** run times on different machines within your team and summarize your observations. 

> Warning: Some of you might not be able to do it on your laptop. It's fine if you're unable to do it. Just make sure you discuss the reasons why you might not have been able to run this on your laptop. 

In [6]:
# adjust based on your os
%cd "figshare_data/"

/Users/raulaguilar/Documents/ubc_courses/block_6/525/525_group_3/notebooks/figshare_data


In [7]:
%%time
# Exclude the "observed_daily_rainfall_SYD.csv" file
file_pattern = re.compile(r"^(?!observed_daily_rainfall_SYD).*\.csv$")

# Extract the model name
def extract_model_name(file_name):
    return re.match(r"([^_]*)", file_name).group(1)

combined_data = []
for file_name in os.listdir("."):
    if file_pattern.match(file_name):
        model_name = extract_model_name(file_name)
        print(model_name)
        data = pd.read_csv(file_name)
        data["model"] = model_name
        combined_data.append(data)

# Combine all the dataframes
combined_data = pd.concat(combined_data, ignore_index=True)
combined_data.to_csv("combined_data.csv", index=False)

MPI-ESM-1-2-HAM
AWI-ESM-1-1-LR
NorESM2-LM
ACCESS-CM2
FGOALS-f3-L
CMCC-CM2-HR4
MRI-ESM2-0
GFDL-CM4
BCC-CSM2-MR
EC-Earth3-Veg-LR
CMCC-ESM2
NESM3
MPI-ESM1-2-LR
ACCESS-ESM1-5
FGOALS-g3
INM-CM4-8
MPI-ESM1-2-HR
TaiESM1
NorESM2-MM
CMCC-CM2-SR5
combined
KIOST-ESM
INM-CM5-0
MIROC6
BCC-ESM1
GFDL-ESM4
CanESM5
SAM0-UNICON
CPU times: user 6min 52s, sys: 29.8 s, total: 7min 22s
Wall time: 8min 25s


In [8]:
combined_data

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM
...,...,...,...,...,...,...,...
124935681,2014-12-27 12:00:00,-30.157068,-29.214660,153.1250,154.3750,6.689683e+00,SAM0-UNICON
124935682,2014-12-28 12:00:00,-30.157068,-29.214660,153.1250,154.3750,7.862555e+00,SAM0-UNICON
124935683,2014-12-29 12:00:00,-30.157068,-29.214660,153.1250,154.3750,1.000503e+01,SAM0-UNICON
124935684,2014-12-30 12:00:00,-30.157068,-29.214660,153.1250,154.3750,8.541592e+00,SAM0-UNICON


In [9]:
%%sh
du -sh combined_data.csv

 11G	combined_data.csv


In [10]:
# table for Q4
data = {
    "Team Member": ["Andy Wang", "Samson Bakos", "Raul Aguilar", "Arjun Radhakrishnan"],
    "Operating System": ["Windows 11", "MacOS Ventura 13.2", "MacOS Monterey 12.5.1", "Windows 11"],
    "RAM": ["32GB", "16GB", "8GB", "16GB"],
    "Processor": ["Intel(R) Core(TM) i7-10870H", "Apple M1", "Apple M2", "Intel(R) Core(TM) i7-12700H"],
    "Is SSD": ["Y", "Y", "Y", "Y"],
    "Time taken": ["6min 37s", "3min 55s", "3min 32s", "3 min 38s"]
}
table = pd.DataFrame(data)

display(table)

,Team Member,Operating System,RAM,Processor,Is SSD,Time taken
0,Andy Wang,Windows 11,32GB,Intel(R) Core(TM) i7-10870H,Y,6min 37s
1,Samson Bakos,MacOS Ventura 13.2,16GB,Apple M1,Y,3min 55s
2,Raul Aguilar,MacOS Monterey 12.5.1,8GB,Apple M2,Y,3min 32s
3,Arjun Radhakrishnan,Windows 11,16GB,Intel(R) Core(TM) i7-12700H,Y,3 min 38s



### 5. Load the combined CSV to memory and perform a simple EDA
rubric={correctness:10,reasoning:10}

1. Investigate at least two of the following approaches to reduce memory usage while performing the EDA (e.g., value_counts). Refer to lecture notes [here](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture1.html#some-tactics-to-deal-with-memory-issue).
    - Changing `dtype` of your data
    - Load just columns that we want
    - Loading in chunks
    
2. ***Compare*** run times on different machines within your team and summarize your observations.

In [11]:
%%time

columns = ["lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)", "model"] 
# dropping Time because timestamp not useful for EDA
# keeping the numeric columns and the model column for EDA

df = pd.read_csv("combined_data.csv", usecols = columns)

CPU times: user 38.1 s, sys: 5.27 s, total: 43.4 s
Wall time: 45.7 s


In [12]:
%%time

# Change float 64 to float 32
float64_cols = list(df.select_dtypes(include='float64'))
df[float64_cols] = df[float64_cols].astype('float32')

# create column subsets to simplify computations
numeric_cols= ["lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)"]
cat_cols = ["model"]


CPU times: user 1.48 s, sys: 5.59 s, total: 7.07 s
Wall time: 9.87 s


In [13]:
print(f"Loaded size is {round(df.memory_usage().sum()*1e-9,2)} GB")

Loaded size is 3.5 GB


In [14]:
%%time

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124935686 entries, 0 to 124935685
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float32
 1   lat_max        float32
 2   lon_min        float32
 3   lon_max        float32
 4   rain (mm/day)  float32
 5   model          object 
dtypes: float32(5), object(1)
memory usage: 3.3+ GB
CPU times: user 2.22 ms, sys: 2.84 ms, total: 5.06 ms
Wall time: 17.6 ms


In [15]:
%%time


df[cat_cols].nunique()

CPU times: user 3.54 s, sys: 1.02 s, total: 4.55 s
Wall time: 5.29 s


model    28
dtype: int64

In [16]:
%%time

df[numeric_cols].describe()

CPU times: user 12.7 s, sys: 2.65 s, total: 15.3 s
Wall time: 16.1 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,1.184971e+08,1.249357e+08,1.184971e+08,1.249357e+08,1.184971e+08
mean,-3.310474e+01,-3.197730e+01,1.469064e+02,1.482143e+02,1.901167e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838411e-06
50%,-3.300000e+01,-3.204189e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [17]:
%%time

df[cat_cols].value_counts()

CPU times: user 3.89 s, sys: 730 ms, total: 4.62 s
Wall time: 4.78 s


model           
combined            62467843
MPI-ESM1-2-HR        5154240
TaiESM1              3541230
CMCC-CM2-HR4         3541230
CMCC-CM2-SR5         3541230
CMCC-ESM2            3541230
NorESM2-MM           3541230
SAM0-UNICON          3541153
FGOALS-f3-L          3219300
GFDL-CM4             3219300
GFDL-ESM4            3219300
EC-Earth3-Veg-LR     3037320
MRI-ESM2-0           3037320
BCC-CSM2-MR          3035340
MIROC6               2070900
ACCESS-CM2           1932840
ACCESS-ESM1-5        1610700
INM-CM5-0            1609650
INM-CM4-8            1609650
KIOST-ESM            1287720
FGOALS-g3            1287720
MPI-ESM-1-2-HAM       966420
MPI-ESM1-2-LR         966420
NESM3                 966420
AWI-ESM-1-1-LR        966420
NorESM2-LM            919800
CanESM5               551880
BCC-ESM1              551880
dtype: int64

In [18]:
%%time

df.isnull().sum()

CPU times: user 8.42 s, sys: 586 ms, total: 9.01 s
Wall time: 9.25 s


lat_min          6438600
lat_max                0
lon_min          6438600
lon_max                0
rain (mm/day)    6438600
model                  0
dtype: int64

In [19]:
%%time

df[numeric_cols].corr()

CPU times: user 5.74 s, sys: 6.69 s, total: 12.4 s
Wall time: 16.3 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
lat_min,1.000000,0.979885,-0.002787,-0.007665,-0.009120
lat_max,0.979885,1.000000,-0.005045,0.018706,-0.009025
lon_min,-0.002787,-0.005045,1.000000,0.993943,0.128947
lon_max,-0.007665,0.018706,0.993943,1.000000,0.131360
rain (mm/day),-0.009120,-0.009025,0.128947,0.131360,1.000000


In [20]:
# table for q5
data = {
    "Team Member": ["Samson Bakos", "Raul Aguilar", "Andy Wang", "Arjun Radhakrishnan"],
    "Operating System": ["MacOS Ventura 13.2", "MacOS Monterey 12.5", "Windows 11", "Windows 11"],
    "RAM": ["16GB", "8GB", "32GB", "16GB"],
    "Processor": ["M1", "M2", "Intel(R) Core(TM) i7-10870H", "Intel(R) Core(TM) i7-12700H"],
    "Is SSD": ["Y", "Y", "Y", "Y"],
    "Time taken": ["51.9s", "49.1s", "66.95", "52.1s"]
}
table = pd.DataFrame(data)

display(table)

,Team Member,Operating System,RAM,Processor,Is SSD,Time taken
0,Samson Bakos,MacOS Ventura 13.2,16GB,M1,Y,51.9s
1,Raul Aguilar,MacOS Monterey 12.5,8GB,M2,Y,49.1s
2,Andy Wang,Windows 11,32GB,Intel(R) Core(TM) i7-10870H,Y,66.95
3,Arjun Radhakrishnan,Windows 11,16GB,Intel(R) Core(TM) i7-12700H,Y,52.1s



### 6. Perform a simple EDA in R
rubric={correctness:15,reasoning:10}

1. Choose one of the methods listed below for transferring the dataframe (i.e., the entire dataset) from Python to R, and explain why you opted for this approach instead of the others.
    - [Parquet file](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture2.html#converting-csv-parquet)
    - [Pandas exchange](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture1.html#use-r-and-python-interchangeably)
    - [Arrow exchange](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture2.html#use-r-and-python-interchangeably-with-arrow)
2. Once you have the dataframe in R, perform a simple EDA.


__ANSWER__:

One of the primary factors in our decision to choose `Arrow exchange` over `Parquet file` and `Pandas exchange` is its efficiency when it comes to serialization and de-serialization processes, allowing an optimal use of memory space. Additionally, since `Arrow exchange` uses an unified memory representation, it facilitates seamless interchangeability between multiple programming languages. Moreover, `Arrow R` has good integration with essential R libraries such as `Dplyr`, `Lubridate`, and `String R`. This integration empowers users to perform comprehensive data wrangling operations on large-scale data frames.

In [21]:
# Change to your local path if necessary
import os
os.environ['R_HOME'] = '/Users/raulaguilar/opt/miniconda3/envs/525_dev/lib/R'

In [22]:
%load_ext rpy2.ipython 
# Load R magic in notebook

In [23]:
# Combined data path
combinedcsv = "combined_data.csv"

In [24]:
%%time

# Build `pyarrow dataset`
dataset = ds.dataset(combinedcsv, format="csv")

# Converting the `pyarrow dataset` to a `pyarrow table`
table = dataset.to_table()

# Converting a `pyarrow table` to a `rarrow table`
r_table = pyra.converter.py2rpy(table)

CPU times: user 22.1 s, sys: 2.46 s, total: 24.6 s
Wall time: 23.9 s


In [25]:
%%time
%%R -i r_table

# Check basic data structure
suppressMessages({
  library(dplyr)
})

glimpse <- r_table |>
    glimpse() 
    
print(glimpse)

Table
124,935,686 rows x 7 columns
$ time      <timestamp[s]> 1889-01-01 05:23:24, 1889-01-02 05:23:24, 1889-01-03 …
$ lat_min         <double> -35.43987, -35.43987, -35.43987, -35.43987, -35.43987…
$ lat_max         <double> -33.57462, -33.57462, -33.57462, -33.57462, -33.57462…
$ lon_min         <double> 141.5625, 141.5625, 141.5625, 141.5625, 141.5625, 141…
$ lon_max         <double> 143.4375, 143.4375, 143.4375, 143.4375, 143.4375, 143…
$ `rain (mm/day)` <double> 4.244226e-13, 4.217326e-13, 4.498125e-13, 4.251282e-1…
$ model           <string> "MPI-ESM-1-2-HAM", "MPI-ESM-1-2-HAM", "MPI-ESM-1-2-HA…
Table
124935686 rows x 7 columns
$time <timestamp[s]>
$lat_min <double>
$lat_max <double>
$lon_min <double>
$lon_max <double>
$rain (mm/day) <double>
$model <string>
CPU times: user 114 ms, sys: 94.8 ms, total: 209 ms
Wall time: 1.07 s


In [26]:
%%time
%%R

# Records by model, top 10
model_rows <- r_table |> 
    count(model) |> 
    arrange(desc(n)) |> 
    collect()
    
print(model_rows)

# A tibble: 28 × 2
   model                n
   <chr>            <int>
 1 combined      62467843
 2 MPI-ESM1-2-HR  5154240
 3 CMCC-CM2-HR4   3541230
 4 CMCC-ESM2      3541230
 5 TaiESM1        3541230
 6 NorESM2-MM     3541230
 7 CMCC-CM2-SR5   3541230
 8 SAM0-UNICON    3541153
 9 FGOALS-f3-L    3219300
10 GFDL-CM4       3219300
# ℹ 18 more rows
# ℹ Use `print(n = ...)` to see more rows
CPU times: user 2.12 s, sys: 861 ms, total: 2.98 s
Wall time: 1.09 s


In [27]:
%%R

# Null count for target variable
target_null <- r_table |>
    filter(is.na(`rain (mm/day)`)) |> 
    count() |> 
    pull()
    
cat("There are", target_null, "null registers in 'rain' column.")

There are 6438600 null registers in 'rain' column.

In [28]:
%%time
%%R

# summary statistics for numeric columns
not_null <- r_table |>
    select(lat_min, lat_max, lon_min, lon_max, `rain (mm/day)`) |> 
    filter(!is.na(`rain (mm/day)`)) |> 
    collect()
        
max_values <- sapply(not_null, max)
min_values <- sapply(not_null, min)
mean_values <- sapply(not_null, mean)
sd_values <- sapply(not_null, sd)

print("Mean values:")
print(mean_values)
print("Min values:")
print(min_values)
print("Max values:")
print(max_values)
print("Sd values:")
print(sd_values)

[1] "Mean values:"
      lat_min       lat_max       lon_min       lon_max rain (mm/day) 
    -33.10482     -31.92201     146.90590     148.28781       1.90117 
[1] "Min values:"
      lat_min       lat_max       lon_min       lon_max rain (mm/day) 
-3.646739e+01 -3.553233e+01  1.406250e+02  1.418750e+02 -3.807373e-12 
[1] "Max values:"
      lat_min       lat_max       lon_min       lon_max rain (mm/day) 
    -29.90000     -27.90606     153.75000     155.62500     432.93951 
[1] "Sd values:"
      lat_min       lat_max       lon_min       lon_max rain (mm/day) 
     1.963549      1.976543      3.793784      3.808084      5.585735 
CPU times: user 25.4 s, sys: 11.4 s, total: 36.8 s
Wall time: 51.1 s


## Specific expectations for this milestone 

- In this milestone, we are looking for a well-documented and self-explanatory notebook that explores different options to tackle big data on your laptop.
- Please discuss any challenges or difficulties you faced when dealing with this large amount of data on your laptop. You can stop combining the data if it takes more than 30 minutes. Briefly explain your approach to overcoming the challenges or reasons why you could not overcome them.
- For questions 5 and 6, you are free to choose any exploratory data analysis (EDA) task you want. Visualization is not necessary; summarizing the data is enough. However, if you want to install additional packages for visualization that are not included in the .yml file, feel free to install them on top of your notebook. If you want to install packages in R, you can do so using `install.packages("dplyr")` under `%%R` magic cell.
- If someone in your team is facing issues with using R in a Python notebook, you can ignore it, as you will not need it for any other milestones. The main purpose of showing it in the lecture was to introduce and get a feel for the serialization and deserialization concept.
- You only need to ***compare*** the time with other team members for questions 4 and 5. You do not need to do this for question 6. You can use the following table to record your results. Feel free to add any other relevant columns.


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Member 1    |                  |     |           |        |            |
| Member 2    |                  |     |           |        |            |
| Member 3    |                  |     |           |        |            |
| Member 4    |                  |     |           |        |            |

## Submission instructions
rubric={mechanics:5}

In the textbox provided on Canvas for the Milestone 1 assignment include:

- The GitHub URL to your notebook.

As comment include
- Repo link
- Teamwork contract